In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Markdown as md, Latex, Math

import statsmodels.api as sm
from patsy import dmatrices
import yfinance as yf
from datetime import datetime

import wrds
connection = wrds.Connection()

%matplotlib inline
plt.style.use("bmh")

Loading library list...
Done


# Read CCM Fundamentals Annual

In [108]:
ccm_fundamentals_annual = pd.read_csv("df_fa.csv")
# ccm_fundamentals_annual["fyear"] = ccm_fundamentals_annual["fyear"].astype("int")
ccm_fundamentals_annual

,GVKEY,LPERMNO,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,cusip,curcd,csho,xrd,exchg,costat,sich,prcc_c,prcc_f,sic
0,1000,25881,19761231,1976.0,INDL,C,D,STD,AE.2,000032102,USD,2.207,NaN,12,I,NaN,5.7500,5.7500,3089.0
1,1000,25881,19771231,1977.0,INDL,C,D,STD,AE.2,000032102,USD,2.226,NaN,12,I,NaN,9.2500,9.2500,3089.0
2,1001,10015,19831231,1983.0,INDL,C,D,STD,AMFD.,000165100,USD,3.568,0.000,14,I,NaN,7.2500,7.2500,5812.0
3,1001,10015,19841231,1984.0,INDL,C,D,STD,AMFD.,000165100,USD,3.568,NaN,14,I,NaN,3.7500,3.7500,5812.0
4,1001,10015,19851231,1985.0,INDL,C,D,STD,AMFD.,000165100,USD,3.988,0.000,14,I,NaN,10.1250,10.1250,5812.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276002,332115,80577,20201231,2020.0,INDL,C,D,STD,ARMP,04216R102,USD,18.688,14.444,12,A,2836.0,2.9849,2.9849,2836.0
276003,335466,14756,20191231,2019.0,INDL,C,D,STD,HOFSQ,G4511M108,USD,25.662,NaN,19,A,4400.0,1.0173,1.0173,NaN
276004,339965,19654,20210131,2020.0,INDL,C,D,STD,SNOW,833445109,USD,287.918,237.946,11,A,7370.0,281.4000,272.4500,7370.0
276005,345920,20194,20201231,2020.0,INDL,C,D,STD,HYFM,44888K209,USD,33.500,NaN,14,A,3524.0,52.5800,52.5800,3524.0


In [109]:
# SIC Codes
ccm_fundamentals_annual["sic"] = ccm_fundamentals_annual[["sic","sich"]].apply(lambda row: (row["sich"] if np.isnan(row["sic"]) else row["sic"]), axis =1)
ccm_fundamentals_annual = ccm_fundamentals_annual.drop("sich", axis = 1)
ccm_fundamentals_annual

,GVKEY,LPERMNO,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,cusip,curcd,csho,xrd,exchg,costat,prcc_c,prcc_f,sic
0,1000,25881,19761231,1976.0,INDL,C,D,STD,AE.2,000032102,USD,2.207,NaN,12,I,5.7500,5.7500,3089.0
1,1000,25881,19771231,1977.0,INDL,C,D,STD,AE.2,000032102,USD,2.226,NaN,12,I,9.2500,9.2500,3089.0
2,1001,10015,19831231,1983.0,INDL,C,D,STD,AMFD.,000165100,USD,3.568,0.000,14,I,7.2500,7.2500,5812.0
3,1001,10015,19841231,1984.0,INDL,C,D,STD,AMFD.,000165100,USD,3.568,NaN,14,I,3.7500,3.7500,5812.0
4,1001,10015,19851231,1985.0,INDL,C,D,STD,AMFD.,000165100,USD,3.988,0.000,14,I,10.1250,10.1250,5812.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276002,332115,80577,20201231,2020.0,INDL,C,D,STD,ARMP,04216R102,USD,18.688,14.444,12,A,2.9849,2.9849,2836.0
276003,335466,14756,20191231,2019.0,INDL,C,D,STD,HOFSQ,G4511M108,USD,25.662,NaN,19,A,1.0173,1.0173,4400.0
276004,339965,19654,20210131,2020.0,INDL,C,D,STD,SNOW,833445109,USD,287.918,237.946,11,A,281.4000,272.4500,7370.0
276005,345920,20194,20201231,2020.0,INDL,C,D,STD,HYFM,44888K209,USD,33.500,NaN,14,A,52.5800,52.5800,3524.0


In [110]:
def get_rnd(xrd_list: pd.Series)->pd.Series:
    '''Function to calculate R&D investment over the last 5 years'''
    return xrd_list.rolling(window=5).apply(lambda x: (np.array([0.2, 0.4, 0.6, 0.8, 0.1]) @ np.array(x)))
# get_rnd(yearly_xrd.iloc[-1]).tail(10)

In [111]:
yearly_xrd = ccm_fundamentals_annual.groupby(["LPERMNO","fyear"])["xrd"].mean().unstack()
yearly_xrd

yearly_rcd = yearly_xrd.apply(lambda row: (get_rnd(row)), axis = 1)
yearly_rcd

fyear,1976.0,1977.0,1978.0,1979.0,1980.0,1981.0,1982.0,1983.0,1984.0,1985.0,...,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0
LPERMNO,,,,,,,,,,,,,,,,,,,,,
10000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93432,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93433,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8.6918,NaN,NaN,NaN,NaN,NaN,NaN
93434,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.8585,1.3526,2.5440,4.0860,5.3970,7.0405,9.8504


In [136]:
def assign_quantile(xrd_list: pd.Series)->pd.Series:
    '''Function to assign each LPERMNO to a quantile depending on its R&D Investment'''
    try:
        arr = pd.qcut(xrd_list, q = 6, labels= ["Low",2,3,4,"High"], duplicates= "drop")
        arr = arr.cat.add_categories('Non R&D')
        arr.loc[pd.isna(arr)] = "Non R&D"
    except:
        arr = pd.qcut(xrd_list, q = 5, labels= ["Low",2,3,4,"High"], duplicates= "drop")
        arr = arr.cat.add_categories('Non R&D')
        arr.loc[pd.isna(arr)] = "Non R&D"
    return arr

# assign_quantile(yearly_rcd[1989.0]).value_counts()

In [138]:
yearly_rcd_class = yearly_xrd.apply(lambda row: (assign_quantile(row)))
yearly_rcd_class

fyear,1976.0,1977.0,1978.0,1979.0,1980.0,1981.0,1982.0,1983.0,1984.0,1985.0,...,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0
LPERMNO,,,,,,,,,,,,,,,,,,,,,
10000,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,...,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D
10001,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,...,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D
10002,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,...,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D
10003,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,...,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D
10005,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,...,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93432,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,...,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D
93433,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,...,3,Low,Low,Low,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D
93434,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,Non R&D,...,Low,Low,Low,Low,Low,2,Low,Low,2,2


# Read CRSP Monthly Stock data

In [167]:
def convert_to_date(date):
    return pd.to_datetime(date[:4]+"/"+date[4:6]+"/"+date[6:]).date()

convert_to_date("19721231")

datetime.date(1972, 12, 31)

In [199]:
from tqdm.notebook import tqdm
tqdm.pandas()

crsp_monthly_data = pd.read_csv("df_ms.csv", low_memory= False)
print("Parsing Dates")
crsp_monthly_data["date"] = crsp_monthly_data["date"].astype("str").progress_apply(lambda x: convert_to_date(x))

print("Add Year")
crsp_monthly_data["year"] = crsp_monthly_data["date"].progress_apply(lambda x: x.year)

print("Add Month")
crsp_monthly_data["month"] = crsp_monthly_data["date"].progress_apply(lambda x: x.month)

crsp_monthly_data

Parsing Dates


  0%|          | 0/3823198 [00:00<?, ?it/s]

In [ ]:
# B in RET occours when none of the PERMNO entries have any price. Thus no return can exist. Thus, dropping
crsp_monthly_data = crsp_monthly_data.drop(crsp_monthly_data[crsp_monthly_data["RET"] == "B"].index).dropna(subset=["RET"]).reset_index(drop = True)

# C in RET occours when the previous price does not exist. thus returns can not be calculated.
crsp_monthly_data["RET"] = crsp_monthly_data["RET"].replace("C",0.0).astype("float")
crsp_monthly_data["RET"]

1                  C
2          -0.257143
3           0.365385
4          -0.098592
5          -0.222656
             ...    
3823193     0.741452
3823194    -0.139087
3823195    -0.095499
3823196     0.462736
3823197     0.243252
Name: RET, Length: 3705088, dtype: object

In [ ]:
crsp_monthly_data

0

# WRDS

In [2]:
connection.list_libraries()

['aha',
 'aha_annual_survey_recent',
 'aha_hcris_recent',
 'aha_it_survey_recent',
 'aha_sample',
 'ahasamp',
 'audit',
 'audit_audit_comp',
 'audit_common',
 'audit_corp_legal',
 'auditsmp',
 'auditsmp_all',
 'bank',
 'blab',
 'block',
 'block_all',
 'boardex',
 'boardex_na',
 'boardex_trial',
 'boardsmp',
 'bvd',
 'bvd_ama_large',
 'bvd_ama_medium',
 'bvd_ama_small',
 'bvd_ama_verylarge',
 'bvdsamp',
 'calcbench_trial',
 'calcbnch',
 'cboe',
 'cboe_all',
 'centris',
 'ciq',
 'ciq_capstrct',
 'ciq_common',
 'ciq_keydev',
 'ciq_pplintel',
 'ciqsamp',
 'ciqsamp_common',
 'ciqsamp_transcripts',
 'cisdm',
 'cisdmsmp',
 'comp',
 'comp_bank',
 'comp_bank_daily',
 'comp_execucomp',
 'comp_global',
 'comp_global_daily',
 'comp_na_annual_all',
 'comp_na_daily_all',
 'comp_na_monthly_all',
 'comp_segments_hist',
 'comp_segments_hist_daily',
 'compa',
 'compb',
 'compbd',
 'compdcur',
 'compg',
 'compgd',
 'comph',
 'compm',
 'compmcur',
 'compnad',
 'compsamp',
 'compsamp_snapshot',
 'compseg',

In [19]:
library = "crspa"
connection.list_tables(library)

['acti',
 'asia',
 'asib',
 'asic',
 'asio',
 'asix',
 'bmdebt',
 'bmheader',
 'bmpaymts',
 'bmquotes',
 'bmyield',
 'bndprt06',
 'bndprt12',
 'bxcalind',
 'bxdlyind',
 'bxmthind',
 'bxquotes',
 'bxyield',
 'ccm_lookup',
 'ccm_qvards',
 'ccmxpf_linktable',
 'ccmxpf_lnkhist',
 'ccmxpf_lnkrng',
 'ccmxpf_lnkused',
 'comphead',
 'comphist',
 'compmaster',
 'crsp_daily_data',
 'crsp_header',
 'crsp_monthly_data',
 'crsp_names',
 'crsp_ziman_daily_index',
 'crsp_ziman_monthly_index',
 'cs20yr',
 'cs5yr',
 'cs90d',
 'cst_hist',
 'dport1',
 'dport2',
 'dport3',
 'dport4',
 'dport5',
 'dport6',
 'dport7',
 'dport8',
 'dport9',
 'dsbc',
 'dsbo',
 'dse',
 'dse62',
 'dse62delist',
 'dse62dist',
 'dse62exchdates',
 'dse62names',
 'dse62nasdin',
 'dse62shares',
 'dseall',
 'dseall62',
 'dsedelist',
 'dsedist',
 'dseexchdates',
 'dsenames',
 'dsenasdin',
 'dseshares',
 'dsf',
 'dsf62',
 'dsfhdr',
 'dsfhdr62',
 'dsi',
 'dsi62',
 'dsia',
 'dsib',
 'dsic',
 'dsio',
 'dsir',
 'dsix',
 'dsiy',
 'dsp500',


In [21]:
table = "ccmxpf_lnkused"
tbl = connection.get_table(library, table, obs=5)
print(tbl.columns)

InternalError: (psycopg2.errors.RaiseException)  'crspa.ccmxpf_lnkused' view is deprecated, please use 'crsp.ccmxpf_lnkused' or 'crsp_a_ccm.ccmxpf_lnkused' 
CONTEXT:  PL/pgSQL function crspa_err(text,text) line 3 at RAISE

[SQL: SELECT * FROM crspa.ccmxpf_lnkused  LIMIT 5 OFFSET 0;]
(Background on this error at: https://sqlalche.me/e/14/2j85)

In [14]:
"liid" in tbl.columns

False